In [3]:
#import the needed libraries
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import os

from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics
from sklearn import datasets
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV



In [4]:
#get and clean stats datset
stats = pd.read_csv("/home/lemina/Desktop/data/stats.csv")
stats_clean = stats.dropna()
stats_clean.columns
#we will use stats.csv to determine some information about the performance of every player in the previous matches
del stats_clean['Unnamed: 0']
del stats_clean['MT']
stats_clean.head(3)

,ID1,ID2,ID_T,ID_R,FS_1,FSOF_1,ACES_1,DF_1,UE_1,W1S_1,...,NA_2,NAOF_2,TPW_2,FAST_2,A1S_2,A2S_2,RPW_1,RPWOF_1,RPW_2,RPWOF_2
312,7042,2992,8885,1,36.0,43.0,3.0,3.0,11.0,29,...,7.0,19.0,30,208.0,188.0,150.0,27.0,46.0,11.0,43.0
314,9094,9831,8885,1,63.0,107.0,6.0,4.0,47.0,43,...,11.0,17.0,95,198.0,177.0,140.0,34.0,88.0,41.0,107.0
315,8146,603,8885,1,62.0,84.0,12.0,5.0,43.0,48,...,15.0,25.0,83,187.0,162.0,134.0,38.0,95.0,26.0,84.0


In [5]:
#we look for the number of times a winner player (colomn ID1 ) had won in the past

from pandas import *
winners =stats_clean['ID1'].value_counts()

df=winners.to_frame(name="count_Won").reset_index()

winnersDF = df.rename(columns={'index': 'ID'})
winnersDF.head(3)

,ID,count_Won
0,5992,97
1,1075,76
2,19,68


In [6]:
#we look for the number of times a looser player (colomn ID2 ) had lost in the past
loosers =stats_clean['ID2'].value_counts()
df=loosers.to_frame(name="count_Loss").reset_index()
loosersDF = df.rename(columns={'index': 'ID'})
loosersDF.head(3)

,ID,count_Loss
0,831,17
1,7,16
2,11522,16


In [7]:
#merge so that we can find for ever player  how many  times  he won and lost 
result = pd.merge(loosersDF,winnersDF,on=['ID'])

result.head(3)


,ID,count_Loss,count_Won
0,831,17,54
1,7,16,13
2,11522,16,21


In [8]:
#here we consider every winner and look for the average of  their good breaks out of the breaks he obtained
stats_clean['wonBreaks'] = stats_clean['RPW_1']/stats_clean['RPWOF_1']*100
dfWon = stats_clean[['ID1','wonBreaks']] 
dfWon = dfWon.rename(columns={'ID1': 'ID'})
avg_wonBreak = dfWon['wonBreaks'].groupby(dfWon['ID']).mean()
avg_wonBreak

frame7=avg_wonBreak.to_frame(name="avg_wonBreakW").reset_index()

frame7.head(3)

,ID,avg_wonBreakW
0,7,42.142244
1,10,50.925926
2,12,38.857143


In [9]:
#here we consider every looser and look for the average of  their good breaks out of the breaks he obtained
stats_clean['wonBreaksL'] = stats_clean['RPW_2']/stats_clean['RPWOF_2']*100
dfLost = stats_clean[['ID2','wonBreaksL']] 
dfLost = dfLost.rename(columns={'ID2': 'ID'})
avg_lostBreak = dfLost['wonBreaksL'].groupby(dfWon['ID']).mean()
avg_lostBreak

frame8=avg_lostBreak.to_frame(name="avg_wonBreakL").reset_index()



#merge to know the number of good breaks a player had done (in the two  cases: when he won and lost)  

result1 = pd.merge(frame7,frame8,on=['ID'])
result1.head(3)

,ID,avg_wonBreakW,avg_wonBreakL
0,7,42.142244,34.793196
1,10,50.925926,33.823529
2,12,38.857143,34.969325


In [10]:
#look for the number of times a winner had won points from the ones he had played 
stats_clean['WonPoints'] = stats_clean['RPW_1']/stats_clean['RPWOF_1']*100


dfWonPoints = stats_clean[['ID1','WonPoints']] 
dfWonPoints = dfWonPoints.rename(columns={'ID1': 'ID'})
avg_Wonpoints_perUser = dfWonPoints['WonPoints'].groupby(dfWonPoints['ID']).mean()
avg_Wonpoints_perUser

frame1=avg_Wonpoints_perUser.to_frame(name="avg_pointsW").reset_index()

frame1.head(3)

,ID,avg_pointsW
0,7,42.142244
1,10,50.925926
2,12,38.857143


In [11]:
#look for the number of times a looser had won points 
stats_clean['LostPoints'] = stats_clean['RPW_2']/stats_clean['RPWOF_2']*100
dfLostPoints = stats_clean[['ID2','LostPoints']] 
dfLostPoints = dfLostPoints.rename(columns={'ID2': 'ID'})
avg_lostpoints_perUser = dfLostPoints['LostPoints'].groupby(dfLostPoints['ID']).mean()
avg_lostpoints_perUser

frame2=avg_lostpoints_perUser.to_frame(name="avg_pointsL").reset_index()
frame2.head(3)


,ID,avg_pointsL
0,7,35.745448
1,10,31.405229
2,12,43.220339


In [12]:
#merge to know each player performance: won points whether he won or failed
resultPoints = pd.merge(frame1,frame2,on=['ID'])
resultPoints.head(3)

,ID,avg_pointsW,avg_pointsL
0,7,42.142244,35.745448
1,10,50.925926,31.405229
2,12,38.857143,43.220339


In [13]:
#include this information in the created dataframe
d = pd.merge(result,resultPoints,on=['ID'])
DF = pd.merge(d,result1,on=['ID'])
DF.head(3)

,ID,count_Loss,count_Won,avg_pointsW,avg_pointsL,avg_wonBreakW,avg_wonBreakL
0,831,17,54,42.015088,29.688267,42.015088,28.298502
1,7,16,13,42.142244,35.745448,42.142244,34.793196
2,11522,16,21,35.161828,27.257577,35.161828,30.387566


In [14]:
#caluclate the averages of double faults and direct errors of every winner
stats_clean['FaultsW'] = (stats_clean['DF_1']+stats_clean['UE_1']).mean()
faults1 = stats_clean[['ID1','FaultsW']] 
faults1 = faults1.rename(columns={'ID1': 'ID'})
faults_perUser =faults1['FaultsW'].groupby(faults1['ID']).mean()
faults_perUser

frame5=faults_perUser.to_frame(name="avgFaultsW").reset_index()
frame5.head(3)


,ID,avgFaultsW
0,7,33.3703
1,10,33.3703
2,12,33.3703


In [15]:
#caluclate the averages of dauble faults and direct errors of every looser
stats_clean['FaultsL'] = stats_clean['DF_2']+stats_clean['UE_2']
faults2 = stats_clean[['ID2','FaultsL']] 
faults2 = faults2.rename(columns={'ID2': 'ID'})
faultsl_perUser =faults2['FaultsL'].groupby(faults2['ID']).mean()
faultsl_perUser

frame6=faultsl_perUser.to_frame(name="avgFaultsL").reset_index()
frame6.head(3)


,ID,avgFaultsL
0,7,52.25
1,10,26.50
2,12,37.00


In [16]:
#merge to know how many errors a player  had done (in both cases win/lose)
FaultPoints = pd.merge(frame5,frame6,on=['ID'])
FaultPoints.head(3)

,ID,avgFaultsW,avgFaultsL
0,7,33.3703,52.25
1,10,33.3703,26.50
2,12,33.3703,37.00


In [17]:
#include this new calculated feature
Data = pd.merge(DF,FaultPoints,on=['ID'])
Data.head(3)

,ID,count_Loss,count_Won,avg_pointsW,avg_pointsL,avg_wonBreakW,avg_wonBreakL,avgFaultsW,avgFaultsL
0,831,17,54,42.015088,29.688267,42.015088,28.298502,33.3703,43.176471
1,7,16,13,42.142244,35.745448,42.142244,34.793196,33.3703,52.250000
2,11522,16,21,35.161828,27.257577,35.161828,30.387566,33.3703,31.625000


In [18]:

#here we consider every winner and look for the average of  their good serves out of the serves he played 

stats_clean['goodServe'] = stats_clean['FS_1']/stats_clean['FSOF_1']*100


dfgoodServe = stats_clean[['ID1','goodServe']] 
dfgoodServe= dfgoodServe.rename(columns={'ID1': 'ID'})

avg_goodServe_perUser = dfgoodServe['goodServe'].groupby(dfgoodServe['ID']).mean()
avg_goodServe_perUser 
frame3=avg_goodServe_perUser.to_frame(name="avg_goodServeW").reset_index()
frame3.head(3)

,ID,avg_goodServeW
0,7,52.946974
1,10,52.941176
2,12,56.441718


In [19]:
#here we consider every looser and look for the average of  their good serves out of the serves he played
stats_clean['lostServe'] = stats_clean['FS_2']/stats_clean['FSOF_2']*100
dfLost = stats_clean[['ID2','lostServe']] 
dfLost = dfLost.rename(columns={'ID2': 'ID'})
avg_lostServe = dfLost['lostServe'].groupby(dfLost['ID']).mean()
avg_lostServe

frame4=avg_lostServe.to_frame(name="avg_goodServeL").reset_index()

frame4.head(3)

,ID,avg_goodServeL
0,7,53.071159
1,10,61.616162
2,12,59.872611


In [20]:
#we merge to see for every player the % of his good serves, whether winner or looser
resultServe = pd.merge(frame3,frame4,on=['ID'])
resultServe.head(3)

,ID,avg_goodServeW,avg_goodServeL
0,7,52.946974,53.071159
1,10,52.941176,61.616162
2,12,56.441718,59.872611


In [21]:
#we add this calculated feature to our final created statistical dataset
finalDataSet = pd.merge(Data,resultServe,on=['ID'])
finalDataSet.head(3)

#these statistics were calculated from the colomns of stats.csv 

#for each player : 
 #   count_loss : how many times he lost in the past
  #  count_won: how many times he won in the past
  # avg_pointsW :  the average of points got when he won
  # avg_pointsL : the average of points got when he lost   
  # avg_wonBreakW : the average of good breaks he made when he won
  # avg_wonBreakL : the average of good breaks he made when he lost  
  # avgFaultsW : average of faults made when he won 
  # avgFaultsL : average of faults made when lost 
  # avg_goodServeW :   average of good serves from the total serves he made when he won 
  # avg_goodServeL : average of good serves from the total serves he made when he lost the match    



,ID,count_Loss,count_Won,avg_pointsW,avg_pointsL,avg_wonBreakW,avg_wonBreakL,avgFaultsW,avgFaultsL,avg_goodServeW,avg_goodServeL
0,831,17,54,42.015088,29.688267,42.015088,28.298502,33.3703,43.176471,58.043741,59.224293
1,7,16,13,42.142244,35.745448,42.142244,34.793196,33.3703,52.250000,52.946974,53.071159
2,11522,16,21,35.161828,27.257577,35.161828,30.387566,33.3703,31.625000,68.678388,68.794538


In [22]:
#get and clean train.csv
train = pd.read_csv("/home/lemina/Desktop/data/train.csv")
train_clean = train.dropna()

del train_clean['Unnamed: 0']
del train_clean['RESULT_G']
train_clean.head(3)


,ID1_G,ID2_G,ID_T_G,ID_R_G,DATE_G
0,17613,18854,8887,4,2012-01-31 00:00:00
1,25192,25191,8888,4,2012-01-31 00:00:00
2,918,10828,8957,4,2012-01-31 00:00:00


In [24]:
# retrieve information of the winning and loosing players from our created statistical dataset

train_clean = train_clean.rename(columns={'ID2_G': 'ID'})

trclean = pd.merge(train_clean,finalDataSet,on=['ID'],how='inner')
trclean= trclean.rename(columns={'ID': 'ID_2'})
trclean.columns
traincl = trclean.rename(columns={'ID1_G': 'ID'})
trainingDataSet = pd.merge(traincl,finalDataSet,on=['ID'],how='inner',suffixes=('_1', '_2'))
trainingDataSet=trainingDataSet.rename(columns={'ID': 'ID_1'})
trainingDataSet.head(3)


,ID_1,ID_2,ID_T_G,ID_R_G,DATE_G,count_Loss_1,count_Won_1,avg_pointsW_1,avg_pointsL_1,avg_wonBreakW_1,...,count_Loss_2,count_Won_2,avg_pointsW_2,avg_pointsL_2,avg_wonBreakW_2,avg_wonBreakL_2,avgFaultsW_2,avgFaultsL_2,avg_goodServeW_2,avg_goodServeL_2
0,563,13796,8887,4,2012-01-31 00:00:00,2,1,47.500000,35.087232,47.500000,...,10,4,41.642139,29.319713,41.642139,34.107893,33.3703,42.6,53.74231,54.943174
1,563,7826,11846,5,2015-01-07 00:00:00,2,1,38.679245,37.828115,38.679245,...,10,4,41.642139,29.319713,41.642139,34.107893,33.3703,42.6,53.74231,54.943174
2,563,578,10807,4,2014-02-03 00:00:00,11,5,39.414202,30.354828,39.414202,...,10,4,41.642139,29.319713,41.642139,34.107893,33.3703,42.6,53.74231,54.943174


In [26]:
#our trainingset has column for winner and column for looser 
#we are going to change this to player1 and player2 with the correspondant stats to train the model
# we will devide the training set into two subsets 
#subset A has the winner in first column and looser in second column (like the oringinal dataset) result = 1 (Player1 won)
#subset B has the winner in second column and looser in  first column  result = 2 (player 2 won)
#this will help us make the classification for future challenges (whether 1 or 2 )


subsetA = trainingDataSet[0:len(trainingDataSet.index)/2]
subsetA
subsetA.insert(4,'Result',1)
subsetA = subsetA.rename(columns={'ID_1': 'Player1' , 'ID_2':'Player2'})


subsetB= trainingDataSet[len(trainingDataSet.index)/2:len(trainingDataSet.index)]

subsetB1 = subsetB[['ID_2', 'count_Loss_1','DATE_G','ID_T_G','ID_R_G','count_Won_1','avg_pointsW_1','avg_pointsL_1','avg_wonBreakW_1','avg_wonBreakL_1',
             'avgFaultsW_1','avgFaultsL_1','avg_goodServeW_1','avg_goodServeL_1','ID_1','count_Loss_2','count_Won_2','avg_pointsW_2',
             'avg_pointsL_2','avg_wonBreakW_2','avg_wonBreakL_2','avgFaultsW_2','avgFaultsL_2','avg_goodServeW_2','avg_goodServeL_2']]

subsetB1.insert(4,'Result',2)
subsetB1 = subsetB1.rename(columns={'ID_1': 'Player2' , 'ID_2':'Player1', 'count_Loss_1' : 'count_Loss_2','count_Won_1': 'count_Won_2',
                                   'avg_pointsW_1':'avg_pointsW_2','avg_pointsL_1':'avg_pointsL_2','avg_wonBreakW_1':'avg_wonBreakW_2',
                                   'avg_wonBreakL_1':'avg_wonBreakL_2','avgFaultsW_1':'avgFaultsW_2','avgFaultsL_1':'avgFaultsL_2',
                                   'avg_goodServeW_1':'avg_goodServeW_2','avg_goodServeL_1':'avg_goodServeL_2','count_Loss_2':'count_Loss_1','count_Won_2':'count_Won_1',
                                   'avg_pointsW_2':'avg_pointsW_1',
             'avg_pointsL_2':'avg_pointsL_1','avg_wonBreakW_2':'avg_wonBreakW_1','avg_wonBreakL_2':'avg_wonBreakL_1','avgFaultsW_2':'avgFaultsW_1','avgFaultsL_2':'avgFaultsL_1','avg_goodServeW_2':'avg_goodServeW_1','avg_goodServeL_2':'avg_goodServeL_1'})
subsetB1

frames = [subsetA, subsetB1]
Trainingset = pd.concat(frames)
Trainingset.drop_duplicates(inplace = True)
#we get out final TrainingSet
TrainingSet =Trainingset.reset_index(drop=True)
TrainingSet.head(3)

,DATE_G,ID_R_G,ID_T_G,Player1,Player2,Result,avgFaultsL_1,avgFaultsL_2,avgFaultsW_1,avgFaultsW_2,...,avg_pointsW_1,avg_pointsW_2,avg_wonBreakL_1,avg_wonBreakL_2,avg_wonBreakW_1,avg_wonBreakW_2,count_Loss_1,count_Loss_2,count_Won_1,count_Won_2
0,2012-01-31 00:00:00,4,8887,563,13796,1,62.000000,42.6,33.3703,33.3703,...,47.500000,41.642139,38.666667,34.107893,47.500000,41.642139,2,10,1,4
1,2015-01-07 00:00:00,5,11846,563,7826,1,42.500000,42.6,33.3703,33.3703,...,38.679245,41.642139,40.000000,34.107893,38.679245,41.642139,2,10,1,4
2,2014-02-03 00:00:00,4,10807,563,578,1,39.909091,42.6,33.3703,33.3703,...,39.414202,41.642139,36.256242,34.107893,39.414202,41.642139,11,10,5,4


In [27]:
#get and clean test.csv 
test = pd.read_csv("/home/lemina/Desktop/data/test.csv")
test_clean = test.dropna()
test_clean.head(3)

,ID1_G,ID2_G,ID_T_G,ID_R_G,DATE_G
0,19942,32203,13204,5,2016-05-25 00:00:00
1,14609,20648,12992,5,2016-06-01 00:00:00
2,13576,1718,12993,4,2016-06-01 00:00:00


In [28]:
#retrieve information of the winning and loosing players from our created statistical dataset
#this will make the same dataframe structure for training and test datasets

test_clean = test_clean.rename(columns={'ID2_G': 'ID'})

clean = pd.merge(test_clean,finalDataSet,on=['ID'],how='inner')
clean= clean.rename(columns={'ID': 'ID_2'})
clean.columns
clt = clean.rename(columns={'ID1_G': 'ID'})
testDataSet = pd.merge(clt,finalDataSet,on=['ID'],how='inner',suffixes=('_1', '_2'))
testDataSet=testDataSet.rename(columns={'ID': 'ID_1'})
testDataSet.drop_duplicates(inplace = True)
testDataSet.head(3)


,ID_1,ID_2,ID_T_G,ID_R_G,DATE_G,count_Loss_1,count_Won_1,avg_pointsW_1,avg_pointsL_1,avg_wonBreakW_1,...,count_Loss_2,count_Won_2,avg_pointsW_2,avg_pointsL_2,avg_wonBreakW_2,avg_wonBreakL_2,avgFaultsW_2,avgFaultsL_2,avg_goodServeW_2,avg_goodServeL_2
0,1092,20648,12814,5,2016-05-25 00:00:00,3,1,46.938776,25.896722,46.938776,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
1,1092,4146,12812,10,2016-05-20 00:00:00,7,5,33.350153,25.268781,33.350153,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
2,1092,4146,12814,4,2016-05-23 00:00:00,7,5,33.350153,25.268781,33.350153,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945


In [29]:
#rename ID_1 and ID_2 by Player1 and Player2 
testDataSet = testDataSet.rename(columns={'ID_1': 'Player1' , 'ID_2':'Player2'})

testDataSet.head(3)


,Player1,Player2,ID_T_G,ID_R_G,DATE_G,count_Loss_1,count_Won_1,avg_pointsW_1,avg_pointsL_1,avg_wonBreakW_1,...,count_Loss_2,count_Won_2,avg_pointsW_2,avg_pointsL_2,avg_wonBreakW_2,avg_wonBreakL_2,avgFaultsW_2,avgFaultsL_2,avg_goodServeW_2,avg_goodServeL_2
0,1092,20648,12814,5,2016-05-25 00:00:00,3,1,46.938776,25.896722,46.938776,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
1,1092,4146,12812,10,2016-05-20 00:00:00,7,5,33.350153,25.268781,33.350153,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
2,1092,4146,12814,4,2016-05-23 00:00:00,7,5,33.350153,25.268781,33.350153,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945


In [32]:
#we will use random forests algorithm

#we specify our explanatory variables
predictorsTrain =  TrainingSet[['count_Loss_1','count_Won_1','avg_pointsW_1','avg_pointsL_1',
'avg_wonBreakW_1','avg_wonBreakL_1','avgFaultsW_1','avgFaultsL_1','avg_goodServeW_1','avg_goodServeL_1','count_Loss_2','count_Won_2',
'avg_pointsW_2','avg_pointsL_2','avg_wonBreakW_2','avg_wonBreakL_2','avgFaultsW_2','avgFaultsL_2','avg_goodServeW_2','avg_goodServeL_2']]

#we set our target variable
targetsTrain = TrainingSet.Result

# 60% of the data will be considered for trainin and 40% for validating
pred_train, pred_test, tar_train, tar_test  = train_test_split(predictorsTrain, targetsTrain, test_size=.4 ,random_state=123)

#this will be used testDataSet to predict future matches
predictorsTest =  testDataSet[['count_Loss_1','count_Won_1','avg_pointsW_1','avg_pointsL_1',
'avg_wonBreakW_1','avg_wonBreakL_1','avgFaultsW_1','avgFaultsL_1','avg_goodServeW_1','avg_goodServeL_1','count_Loss_2','count_Won_2',
'avg_pointsW_2','avg_pointsL_2','avg_wonBreakW_2','avg_wonBreakL_2','avgFaultsW_2','avgFaultsL_2','avg_goodServeW_2','avg_goodServeL_2']]



In [34]:
#use grid search to look for best parameters to fit the model


classifier=RandomForestClassifier( n_jobs=-1,max_features= 'sqrt' , oob_score = True ) 

param_grid = { 
           "n_estimators" : [18, 27, 36, 45,54],
           "max_depth" : [10, 15, 20, 25, 30],
           "min_samples_leaf" : [4, 6, 8, 10]}
           

CV_classifier = GridSearchCV(estimator=classifier, param_grid=param_grid, cv= 8)
CV_classifier.fit(pred_train,tar_train) 
print CV_classifier.best_params_


{'n_estimators': 45, 'max_depth': 15, 'min_samples_leaf': 6}


In [36]:
# we find {'n_estimators': 45, 'max_depth': 15, 'min_samples_leaf': 6}
# we fit the model with these hyperparamaeters
rfclassifier = RandomForestClassifier(n_estimators=45, max_depth=15, max_features='sqrt', min_samples_leaf = 6)
rfclassifier=rfclassifier.fit(pred_train,tar_train)
preds=rfclassifier.predict(pred_test)
preds
sklearn.metrics.confusion_matrix(tar_test,preds)
sklearn.metrics.accuracy_score(tar_test, preds)
# the model accuracy on the validation set is 84% / 84% were classified correctly 
#now we will use this classifier to predict future games 
predictions=rfclassifier.predict(predictorsTest)
predictions


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1,
       2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [38]:
unique, counts = np.unique(predictions, return_counts=True)
dict(zip(unique, counts))

{1: 285, 2: 347}

In [39]:
model = ExtraTreesClassifier()
model.fit(pred_train,tar_train)
# display the relative importance of each attribute
print(model.feature_importances_)

[0.05359835 0.05132063 0.04918682 0.03877235 0.05119351 0.04043897
 0.         0.04643847 0.04352927 0.04546728 0.1234915  0.12136193
 0.04842924 0.06184041 0.04288509 0.05316261 0.         0.05067326
 0.03770644 0.04050387]


In [40]:
winner = np.asarray(predictions)
Winner= pd.DataFrame(winner, columns=['Winner'])
Winner.head(3)

,Winner
0,1
1,1
2,1


In [41]:
Test=testDataSet[['Player1','Player2','ID_T_G','ID_R_G','DATE_G']]
Test.head(3)

,Player1,Player2,ID_T_G,ID_R_G,DATE_G
0,1092,20648,12814,5,2016-05-25 00:00:00
1,1092,4146,12812,10,2016-05-20 00:00:00
2,1092,4146,12814,4,2016-05-23 00:00:00


In [42]:
TestSet =Test.join(Winner)
TestSet.columns
TestSet.rename(columns={'0': 'Winner'})
TestSet.columns
TestSet.head(3)

,Player1,Player2,ID_T_G,ID_R_G,DATE_G,Winner
0,1092,20648,12814,5,2016-05-25 00:00:00,1
1,1092,4146,12812,10,2016-05-20 00:00:00,1
2,1092,4146,12814,4,2016-05-23 00:00:00,1


In [43]:
TestSet.to_csv('/home/lemina/Desktop/test.csv', sep=',', encoding='utf-8')